In [11]:
import os
import time
import warnings ; warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [12]:
path = 'C:/Users/bitcamp/Desktop/dacon/'
os.chdir(path)

In [13]:
# 데이터 불러오기
train = pd.read_csv(path + '201901-202003.csv')
train.head(n = 5)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [14]:
train.isnull().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
SEX_CTGO_CD          0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
dtype: int64

In [15]:
train = train.fillna('')
train.isnull().sum()

REG_YYMM        0
CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
dtype: int64

In [17]:
train = train[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
train = train.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop = False)
train

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
9433,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299
9434,202003,충북,한식 음식점업,16152482704
9435,202003,충북,호텔업,15248550
9436,202003,충북,화장품 및 방향제 소매업,428881434


In [19]:
submit = pd.read_csv(path + 'submission.csv', index_col = 0)
submit = submit.loc[submit['REG_YYMM'] == 202004]
submit = submit[['CARD_SIDO_NM', 'STD_CLSS_NM']]
submit

      REG_YYMM CARD_SIDO_NM            STD_CLSS_NM           AMT
id                                                              
0       202004           강원             건강보조식품 소매업  9.605901e+07
1       202004           강원                골프장 운영업  2.915798e+09
2       202004           강원            과실 및 채소 소매업  9.948169e+08
3       202004           강원      관광 민예품 및 선물용품 소매업  1.331730e+07
4       202004           강원   그외 기타 분류안된 오락관련 서비스업  0.000000e+00
...        ...          ...                    ...           ...
1389    202007           충북  피자 햄버거 샌드위치 및 유사 음식점업  1.315245e+09
1390    202007           충북                한식 음식점업  1.615248e+10
1391    202007           충북                    호텔업  1.524855e+07
1392    202007           충북          화장품 및 방향제 소매업  4.288814e+08
1393    202007           충북               휴양콘도 운영업  1.273349e+07

[1394 rows x 4 columns]


,CARD_SIDO_NM,STD_CLSS_NM
id,,
0,강원,건강보조식품 소매업
1,강원,골프장 운영업
2,강원,과실 및 채소 소매업
3,강원,관광 민예품 및 선물용품 소매업
4,강원,그외 기타 분류안된 오락관련 서비스업
...,...,...
692,충북,피자 햄버거 샌드위치 및 유사 음식점업
693,충북,한식 음식점업
694,충북,호텔업


In [22]:
REG_YYMMs = np.sort(train['REG_YYMM'].unique())
REG_YYMMs

array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003],
      dtype=int64)

In [26]:
AMTs = []
for REG_YYMM in REG_YYMMs:
    df = train.loc[train['REG_YYMM'] == REG_YYMM]
    df = df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
    tmp = submit.merge(df, left_on = ['CARD_SIDO_NM', 'STD_CLSS_NM'],
                            right_on = ['CARD_SIDO_NM', 'STD_CLSS_NM'], how = 'left')
    tmp = tmp.fillna(0)
    AMT = list(tmp['AMT'])
    AMTs.append(AMT)
AMTs = np.array(AMTs)
AMTs

array([[1.48354746e+08, 1.22920031e+09, 1.28697911e+09, ...,
        3.46627180e+07, 7.28722041e+08, 1.98574855e+08],
       [1.46715871e+08, 1.15224097e+09, 1.11626588e+09, ...,
        3.39593490e+07, 6.55288626e+08, 1.65872181e+08],
       [1.09439025e+08, 2.76049714e+09, 9.39227950e+08, ...,
        3.60300100e+07, 7.29328422e+08, 1.07431443e+08],
       ...,
       [1.85008768e+08, 1.24110201e+09, 1.57655655e+09, ...,
        4.13298500e+07, 5.64909717e+08, 2.22073112e+08],
       [7.65055610e+07, 9.68456455e+08, 8.67214360e+08, ...,
        1.15169300e+07, 4.95012925e+08, 4.70858300e+07],
       [9.60590120e+07, 2.91579800e+09, 9.94816943e+08, ...,
        1.52485500e+07, 4.28881434e+08, 1.27334900e+07]])